In [3]:
!pip3 install -r ./requirements.txt -q

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [26]:
#pip install openai

In [4]:
pip install python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [37]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)
#os.environ.get('OPENAI_API_KEY')


True

In [14]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain_community.chat_models.openai:

class ChatOpenAI(langchain_core.language_models.chat_models.BaseChatModel)
 |  ChatOpenAI(*, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, client: Any = None, async_client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, api_key: Optional[str] = None, base_url: Optional[str] = None, organization: Optional[str] = None, openai_proxy: Optional[str] = None, timeout: Union[float, Tuple[float, float], Any, NoneType] = None, max_retries: int = 2, streaming: bool = False, n: int = 1, max_tokens: Optional[int] = None, tiktoken_model_name: Optional[str] = Non

### LLM Models (Wrappers):GPT-3

In [16]:
from langchain.llms import OpenAI

In [28]:
llm = OpenAI(model_name='text-davinci-003',temperature=0.7,max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [29]:
output = llm('explain smart contract in one sentence')
print(output)



A smart contract is a self-executing digital agreement stored on a blockchain that is designed to automatically enforce the terms of a contract.


### ChatModels: GPT-3.5-Turbo and GPT-4

In [5]:
from langchain.schema import(AIMessage,HumanMessage,SystemMessage)
from langchain.chat_models import ChatOpenAI

In [8]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5,max_tokens=1024)
messages=[
    SystemMessage(content='You are a physicist and respond only in English.'),
    HumanMessage(content='explain quantum computing in one sentence')
]
output = chat(messages)

In [9]:
print(output)

content='Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data in a fundamentally different way than classical computing.'


### Prompt Templates

In [11]:
from langchain import PromptTemplate

In [12]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)
print(prompt)

input_variables=['language', 'virus'] template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.'


In [13]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct',temperature=0.7)
output = llm(prompt.format(virus='ebola',language='english'))
print(output)



Ebola is a highly virulent and deadly virus that belongs to the family Filoviridae. It was first identified in 1976 in the Democratic Republic of Congo and has since caused several outbreaks in Africa. The virus is transmitted through contact with infected bodily fluids and can lead to severe hemorrhagic fever, with a mortality rate of up to 90%. Despite efforts to control and contain the virus, Ebola remains a significant public health concern and ongoing research is crucial in developing effective treatments and vaccines.


## Caching LLM Responses

### In-Memory Cache

In [22]:
from langchain.globals import set_llm_cache
from langchain_community.chat_models import openai
llm = OpenAI(model='gpt-3.5-turbo-instruct')

In [23]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a computer scientist can understand'
llm.invoke(prompt)

CPU times: user 18.4 ms, sys: 5.37 ms, total: 23.8 ms
Wall time: 865 ms


'\n\nWhy was the computer cold?\n\nBecause it left its Windows open!'

In [27]:
#Second invocation to see the effect of caching
%%time
llm.invoke(prompt)

CPU times: user 603 µs, sys: 71 µs, total: 674 µs
Wall time: 684 µs


'\n\nWhy was the computer cold?\n\nBecause it left its Windows open!'

In [ ]:
## SQLite Caching

from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".lanchain.db")) #this will be the db name
#first request
llm.invoke("Tell me a joke")

#Second request (cached ,faster)
llm.invoke("Tell me a joke")

## Streaming in Langchain

- Process of delivering the response in a continous stream of data instead of sending all at once.
- This allows the user to receive the response piece by piece as it is generated, which improve user experience

In [38]:
from langchain_community.chat_models import ChatOpenAI

In [40]:
llm = ChatOpenAI()
prompt = "Write a rock song about Moon and a Dove."
print(llm.invoke(prompt).content) # Without streaming

Verse 1:
In the stillness of the night
The moon shines bright
A dove takes flight
Through the dark sky's might

Chorus:
Moon and dove, forever entwined
In a dance of light and shadow
Their love will never fade
In the night, they'll always find a way

Verse 2:
The moon's gentle glow
Guides the dove below
Through the darkness they flow
Their love continues to grow

Chorus:
Moon and dove, forever entwined
In a dance of light and shadow
Their love will never fade
In the night, they'll always find a way

Bridge:
Together they soar
Through the night forevermore
Their love is pure
In the moon's allure

Chorus:
Moon and dove, forever entwined
In a dance of light and shadow
Their love will never fade
In the night, they'll always find a way

Outro:
In the sky, they'll always be
Moon and dove, forever free
Their love will never die
In the night, they'll always fly high.


In [35]:
#With streaming
for chunk in llm.stream(prompt):
    print(chunk,end='',flush=True)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-GxjYt***************************************HdTM. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### Simple Chain

In [23]:
# chain allows us to combine multiple input together
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature= 0.7)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

#creates a simple chain that takes in a prompt template.
#formats it with the user input and returns the response from the LLM.
chain = LLMChain(llm=llm,prompt=prompt) 
output = chain.run({'virus':'HSV','language':'english'})


In [24]:
print(output)

HSV, or herpes simplex virus, is a common viral infection that affects humans. It is classified into two types: HSV-1 and HSV-2. HSV-1 is primarily associated with oral herpes, causing cold sores or fever blisters around the mouth and on the face. On the other hand, HSV-2 is mainly responsible for genital herpes, causing painful blisters or sores in the genital area. Both types of HSV can be transmitted through direct contact with an infected individual, and although the virus cannot be cured, antiviral medications can help manage and reduce the frequency of outbreaks.


### Sequential Chain

In [ ]:
With Sequential chain, you can make a series of call to one or more LLMs.
You can take the output from one chain and use it as the input to another chain
- SimpleSequentialChain: Each chain has a single input and single output
- General form of sequential chains